# Linear programming

This notebook gives 
- an introduction to linear programming
- two worked examples
- a practical exercise

## Resources & further reading

- [6 part blog post series on linear programming with PuLP - Ben Alex Keen](http://benalexkeen.com/blog/)
- [Linear programming - Michel Goemans](https://math.mit.edu/~goemans/18310S15/lpnotes310.pdf)
- [Linear programming - Thomas Ferguson](https://www.math.ucla.edu/~tom/LP.pdf)

## Why linear programming

[Linear programming - Wikipedia](https://en.wikipedia.org/wiki/Linear_programming)

Classical optimization method
- minimize cost or maximize profit
- guranteed global optimum
- deterministic
- can be used for prediction or control

I have used LP lots in my career: 
- to mixed integer linear programming in Excel an an energy engineer using [OpenSolver](https://opensolver.org/) (University of Auckland, New Zealand) to optimize the dispatch of combined heat and power
- as a data scientist we developed [a linear program to dispatch electric battery storage](https://github.com/ADGEfficiency/energy-py-linear)

## Why not linear programming

Many business problems are not linear - but if yours is, linear programs are a huge advantage.

Uncertanitity on the inputs - see [robust optimization](https://en.wikipedia.org/wiki/Robust_optimization).

## Linear programming with PuLP

We can do linear programming in Python using [PuLP](https://pythonhosted.org/PuLP/) - a library that abstracts away the mechanics of solving linear programs.  The mathematics of the optimization is not covered here - if you are interested, the [Simplex Method](https://en.wikipedia.org/wiki/Simplex_algorithm) is a good place to start.

## The structure of a linear program

**Objective function** - minimize or maximize

**Variables** - things you can change - continuous, integer, binary

**Constraints** - equality (==) or inequality (>=, <=)

## What do we mean by linear?

If you have a vector of two variables

$\textbf{V} = \begin{bmatrix}v_{1} \\ v_{2} \\ \end{bmatrix}$

You cannot do operations that are non-linear
- multipying a variable by itself $ v_{1}^{2} $
- multipying a variable by another variable $v_{1} * v_{2} $ (bilinearity)

This means that linear programs are limited in scope 
- systems that can be modelled in terms of linear relationships

A linear program is a **convex optimization problem**
- only one globally optimal solution (or infeasible)

## Using linear programs in industry

Writing linear programs requires two skills
1. identifying the business problem can be modelled as a linear program
2. writing the objective, variables and constraints as a program

The first step is harder

You don't need to know how the optimization is done
- driving a car - the detail of combustion is hidden
- stochastic gradient descent (used to train neural nets) - you can optimize a function without knowing exactly how the optimizer works

## Example - diet problem

From [Linear programming - Michel Goemans](https://math.mit.edu/~goemans/18310S15/lpnotes310.pdf).

> In the diet model, a list of available foods is given together with the nutrient content and the cost
per unit weight of each food. A certain amount of each nutrient is required per day. For example,
here is the data corresponding to a civilization with just two types of grains (G1 and G2) and three
types of nutrients (starch, proteins, vitamins)

| | Starch [kg/kg] | Protiens [kg/kg] | Vitamins [kg/kg] | Cost [$/g] |
|---|---|---|---|---|
|apples| 5 | 4 | 2| 0.6 |
|oranges| 7 | 2 | 1| 0.35 |

We have a requirement per day of 8g of starch, 15g of protiens and 3g of vitamins

### Formulating as a linear problem

Let's map this problem directly onto the definition of linear programming

Objective function = minimize cost

Variables = amount of apples & oranges

Constraints = daily requirements of starch, protien and vitamins

### The PuLP api

Below is a full linear program for the problem above:

In [ ]:
!pip install pulp -q

from pulp import LpProblem, LpMinimize, LpVariable, LpStatus

problem = LpProblem('cost minimization', LpMinimize)

apples = LpVariable('apples', 0, None)
bannanas = LpVariable('banannas', 0, None)

#  add the objective function
problem += apples * 0.6 + bannanas * 0.35

#  add contstraints
#  starch
problem += apples * 5 + bannanas * 7 >= 8
#  protien
problem += apples * 4 + bannanas * 2 >= 15
#  vitamins
problem += apples * 2 + bannanas * 1 >= 3

problem.solve()
print(LpStatus[problem.status])

for v in (apples, bannanas):
    print('{} {}'.format(v.name, v.varValue))

## Standard form of a linear program

A linear program is said to be in standard form if
- it is a maximization program
- there are only equalities (no inequalities)
- all variables are restricted to be nonnegative

## Example - resource allocation 

Our company makes two products - wind turbines and solar panels.  We want to purchase a certain amount to meet a given amount of electricity demand, while minimizing carbon impact.
- a certain number of products (our constraints)

Inputs / assumptions are
- the amount of electricity demand we need to meet
- producing one wind turbine requires two units of coal and one unit of steel
- producing one solar panel requires three units of steel

Let's put this into code.  First thing we need to choose is a data structure to hold infomation about our two products.  Let's use a `namedtuple` (a more user friendly `tuple`):

In [ ]:
from collections import namedtuple

Product = namedtuple('Product', ['name', 'coal', 'steel', 'limit'])
wind = Product('wind', 2, 1, 100)
solar = Product('solar', 0, 3, 100)
wind

The `namedtuple` allows us to refer to data via postiton (as with a normal `tuple`) and by name:

In [ ]:
wind[0]

In [ ]:
wind.coal

Now we will model the costs of the resources that make up our products:

In [ ]:
Resource = namedtuple('Resource', ['name', 'cost'])

coal = Resource('coal', 3)
steel = Resource('steel', 10)

Now lets start to use PuLP to build the linear program:

In [ ]:
problem = LpProblem('carbon minimization', LpMinimize)

products = [wind, solar]
variables = [LpVariable(p.name, 0, p.limit) for p in products]

variables

Add the objective function, which is total carbon cost:

In [ ]:
problem += sum(
    [v * p.coal * coal.cost for v, p in zip(variables, products)]
    
) + sum(
    [v * p.steel * steel.cost for v, p in zip(variables, products)]
)

Add the constraint to deliver a certain number of units:

In [ ]:
demand = 10

problem += sum(variables) == demand

In [ ]:
problem.solve()
print(LpStatus[problem.status])

for v in variables:
    print('{} {}'.format(v.name, v.varValue))

## Practical - transportation problem

$P$ ports
- a capacity (num. units) 
- a cost (\$/unit)

$M$ markets
- a demand (num. units)

Map this problem directly onto the definition of linear programming

- objective function = ?
- variables = ?
- constraints = ?

Build a linear program to solve the transportation problem

In [ ]:
port_capacity = [20, 30, 30, 50]
market_demand = [20, 10, 5]

#  one price for a port-market pair
np.random.seed(42)
port_cost = np.random.uniform(
    0, 1, 
    size=len(port_capacity) * len(market_demand)
)

port_cost = port_cost.reshape(len(port_capacity), len(market_demand))

We can access the cost to trade from a port to a market by indexing `port_cost[port, market]`

In [ ]:
#  trade from port 0 to market 0
port_cost[0, 0]

In [ ]:
#  trade from port 3 to market 2
port_cost[3, 2]

In [ ]:
# from answers import transportation
# transportation()